In [25]:
import pandas as pd
from gensim.models import Word2Vec
import jieba

# 1. 读取数据（确保train.csv在同目录）
try:
    df = pd.read_csv('train.csv')
except FileNotFoundError:
    print("❌ 错误：未找到train.csv文件！请将文件放在当前目录，或从学习通/ GitHub重新下载。")
    raise  # 终止执行，避免后续错误

# 2. 检查数据列名（根据你的数据，实际列名为'comment'）
print("数据列名：", df.columns)  # 应输出 Index(['label', 'comment'], ...)
if 'comment' not in df.columns:
    print("❌ 错误：数据中缺少'comment'列！请检查数据是否正确下载。")
    raise

# 3. 数据预处理（中文分词，使用'comment'列）
sentences = []
for text in df['comment']:
    if isinstance(text, str):  # 处理字符串类型
        words = jieba.lcut(text.strip())  # 分词并去除前后空格
        if words:  # 过滤空句子
            sentences.append(words)
    elif pd.isna(text):  # 处理缺失值（如NaN）
        continue  # 跳过空值

# 4. 训练Skip-Gram模型（sg=1，任务1核心要求）
model = Word2Vec(
    sentences=sentences,        # 分词后的句子列表
    vector_size=100,            # 词向量维度（默认100，可自定义）
    window=5,                   # 上下文窗口大小
    min_count=1,                # 忽略词频<1的词（包含低频词）
    sg=1,                       # 使用Skip-Gram模型（1=Skip-Gram，0=CBOW）
    epochs=10,                  # 训练轮数
    workers=4,                  # 并行线程数（根据CPU核心数调整）
    seed=42                     # 固定随机种子，确保结果可复现（可选）
)

# 5. 验证模型训练成功（可选，显示词表大小）
print(f"✅ 模型训练完成，词表大小：{len(model.wv.key_to_index)}")

数据列名： Index(['label', 'comment'], dtype='object')
✅ 模型训练完成，词表大小：11780


In [27]:
# 检查“环境”是否在词表中（避免KeyError）
if "环境" not in model.wv.key_to_index:
    print("❌ 错误：词汇‘环境’未在训练数据中出现！请检查评论中是否包含该词。")
else:
    env_vector = model.wv["环境"]  # 获取词向量
    print("词向量：", env_vector)          # 输出向量数值（维度100）
    print("形状：", env_vector.shape)      # 输出形状 (100,)

词向量： [ 0.22136146 -0.27616668  0.2645627  -0.04798624 -0.67146605  0.5530261
  0.2562593   0.2827301  -0.540008   -0.1930875  -0.93454367  0.47286826
 -0.11612877  0.27612126 -0.32475546 -0.4437183   0.37124655 -0.57307565
 -0.5240218   0.30186728 -0.19834386 -0.18458013 -0.0320374  -0.2038352
  0.19505586 -0.48117423  0.02299462 -0.08084007 -1.2929502  -0.6559462
 -0.28957552  0.16172746  0.09849492 -0.54721636  0.17616266  0.01310606
  0.04639869 -0.04358044 -0.07836898  0.38656718  0.73672974 -0.02542879
  0.90403956 -0.21364231 -0.3506671   0.1512568  -0.02012627  0.4158034
  1.1815318   0.06503085  0.57651347 -0.14549804 -0.13040964 -0.16096403
  0.21315849  0.04191227  0.26670685  0.5338617   0.0452549  -0.33788556
 -0.598697    0.14211692  0.09599409  0.06522573  0.25837758  0.30410844
  0.08417613 -0.7211761  -0.3878452  -0.2894672  -0.44232196  0.14583011
  0.13611801 -0.19276193 -0.17327906  0.09812251 -0.57341135 -0.2936502
  0.3503093  -0.13503322  0.4375291  -0.19763336 -0

In [29]:
# 检查“好吃”是否在词表中
if "好吃" not in model.wv.key_to_index:
    print("❌ 错误：词汇‘好吃’未在训练数据中出现！")
else:
    similar_words = model.wv.most_similar("好吃", topn=3)  # 取前3个相似词
    print("与‘好吃’最接近的3个词：")
    for word, score in similar_words:
        print(f"- {word}（相似度：{score:.4f}）")

与‘好吃’最接近的3个词：
- 入味（相似度：0.7703）
- 油腻（相似度：0.7589）
- 有味（相似度：0.7546）


In [31]:
# 检查所有词汇是否在词表中
missing_words = []
for word in ["好吃", "美味", "蟑螂"]:
    if word not in model.wv.key_to_index:
        missing_words.append(word)
if missing_words:
    print(f"❌ 错误：以下词汇未在词表中：{missing_words}")
else:
    sim1 = model.wv.similarity("好吃", "美味")   # 正向相似度
    sim2 = model.wv.similarity("好吃", "蟑螂")   # 负向相似BB
    print(f"‘好吃’ vs ‘美味’ 相似度：{sim1:.4f}")
    print(f"‘好吃’ vs ‘蟑螂’ 相似度：{sim2:.4f}")

‘好吃’ vs ‘美味’ 相似度：0.6996
‘好吃’ vs ‘蟑螂’ 相似度：0.2809


In [33]:
# 检查运算涉及的词汇是否在词表中
required_words = ["餐厅", "聚会", "安静"]
missing_words = [word for word in required_words if word not in model.wv.key_to_index]
if missing_words:
    print(f"❌ 错误：以下词汇未在词表中，无法运算：{missing_words}")
else:
    result = model.wv.most_similar(
        positive=["餐厅", "聚会"],  # 正向词汇
        negative=["安静"],          # 负向词汇
        topn=1                      # 取1个最相关结果
    )
    print("向量运算结果（餐厅+聚会-安静）：")
    print(f"- 最相关词：{result[0][0]}（相似度：{result[0][1]:.4f}）")

向量运算结果（餐厅+聚会-安静）：
- 最相关词：美好（相似度：0.7610）
